<a href="https://colab.research.google.com/github/JSJeong-me/AI-Innovation-2024/blob/main/NLP/4-7-LangChain-ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-core langgraph>0.2.27

In [2]:
!pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 26.5 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
api_key = os.getenv("OPENAI_API_KEY")

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [5]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ce21fc2a-aa27-4cf8-843f-39f326025ca4-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

Hello Bob! How can I assist you today?


In [6]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, I do not have access to personal information such as your name. How may I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 12, 'total_tokens': 36, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-aa5a501a-4493-4876-9052-14bd1535b9d6-0', usage_metadata={'input_tokens': 12, 'output_tokens': 24, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

## Message persistence

In [7]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

In [8]:


# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

In [9]:



# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


In [10]:


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

In [11]:


# Add memory
memory = MemorySaver()

In [12]:

app = workflow.compile(checkpointer=memory)

In [ ]:
# app.invoke({"messages": [HumanMessage(content="Hi! I'm Bob.")]})

In [13]:
config = {"configurable": {"thread_id": "abc123"}}

In [25]:
query = "A black cat sat on the bench." # A black cat sat on the bench  Hi! I'm Bob.

In [26]:


input_messages = [HumanMessage(query)]

In [27]:

output = app.invoke({"messages": input_messages}, config)

In [28]:
output["messages"]

[HumanMessage(content="Hi! I'm Bob.", additional_kwargs={}, response_metadata={}, id='75c38491-9903-4f47-bc97-10a3c2248166'),
 AIMessage(content='Hello Bob! How are you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 13, 'total_tokens': 21, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9c4b5795-5227-408f-966d-eb8ff07c6c8a-0', usage_metadata={'input_tokens': 13, 'output_tokens': 8, 'total_tokens': 21, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}),
 HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='b88042ca-f6ee-4932-b64d-11ca106f13f3'),
 AIMessage(content='Your name is Bob.', additional_kwargs={'refusal': None}, response_metad

In [29]:

output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

That's an interesting sentence! Is there anything else you'd like to share or talk about?


In [30]:
query = "Whatis on the bench?" # What is on the bench?

In [31]:


input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

A black cat is on the bench.


In [32]:
config = {"configurable": {"thread_id": "abc234"}}

In [33]:


input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm an AI digital assistant, so I do not have physical senses to see or touch objects. Can you please provide me with more information about the bench you are referring to?


In [34]:
config = {"configurable": {"thread_id": "abc123"}}

In [35]:


input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

A black cat is on the bench.


In [36]:
output

{'messages': [HumanMessage(content="Hi! I'm Bob.", additional_kwargs={}, response_metadata={}, id='75c38491-9903-4f47-bc97-10a3c2248166'),
  AIMessage(content='Hello Bob! How are you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 13, 'total_tokens': 21, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9c4b5795-5227-408f-966d-eb8ff07c6c8a-0', usage_metadata={'input_tokens': 13, 'output_tokens': 8, 'total_tokens': 21, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='b88042ca-f6ee-4932-b64d-11ca106f13f3'),
  AIMessage(content='Your name is Bob.', additional_kwargs={'refusal': None}

## Prompt templates

In [53]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a great assistant and from now on, please speak in an elementary school style.",  # You are a great assistant and from now on, please speak in an elementary school style. You talk like a pirate. Answer all questions to the best of your ability.
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [54]:
workflow = StateGraph(state_schema=MessagesState)

In [55]:



def call_model(state: MessagesState):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": response}

In [56]:



workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

In [57]:


memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [58]:
config = {"configurable": {"thread_id": "abc001"}}
query = "Hi! I'm Joon"

In [59]:


input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hi Joon! I'm your assistant. How can I help you today?


In [60]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Joon! That's a cool name!


## 언어 변경

In [61]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a great assistant and from now on, please speak in an elementary school style. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [62]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

In [63]:



class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

In [65]:



workflow = StateGraph(state_schema=State)

In [66]:

def call_model(state: State):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": [response]}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [67]:
config = {"configurable": {"thread_id": "abc002"}}
query = "Hi! I'm Joon."
language = "Korean"

In [68]:
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

안녕! 난 준이야. 무엇을 도와줄까?


In [69]:
query = "What is my name?" # 당신의 이름은? What is my name?

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

당신의 이름은 준이야! :)
